##Importing Libraries

In [ ]:
import numpy as np
import pandas as  pd
import seaborn as sns
import matplotlib.pyplot as plt
from  sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,f1_score,recall_score

##Loading the Datasets

In [ ]:
realdf=pd.read_csv("true.csv")
realdf.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fakedf=pd.read_csv("fake.csv")
fakedf.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


##Dropping unwanted Columns

In [ ]:
realdf.drop("date",axis=1,inplace=True)
fakedf.drop("date",axis=1,inplace=True)

##Adding Target Column "Label" for both df to classify Fake/True

In [ ]:
fakedf['label']=0
realdf['label']=1

##Creating new df by Merging two df (Fakedf & Truedf)

In [ ]:
newdf=pd.concat([fakedf,realdf],axis=0,ignore_index=True)
newdf.head()

,title,text,subject,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,0


In [ ]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   label    44898 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


##Assigning feature & target columns to x & y, and splitting train & test data

In [ ]:
x=newdf.drop('label',axis=1)
y=newdf['label']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)

##Checking if the Target Column labels is balanced or not

In [ ]:
y.value_counts()

,count
label,
0,23481
1,21417


##Creating preprocessors in ColumnTransformer

In [ ]:
preprocess=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),['subject']),
                                           ('vector1',TfidfVectorizer(),'title'),
                                           ('vector2',TfidfVectorizer(),'text')])

##Creating Pipeline for Chaining Preprocessors and **LogisticRegression** Model

In [ ]:
pipe=Pipeline(steps=[('preprocess',preprocess),
                     ('model',LogisticRegression())])

##Training the **LogisticRegression** Model

In [ ]:
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('encoder', OneHotEncoder(),
                                                  ['subject']),
                                                 ('vector1', TfidfVectorizer(),
                                                  'title'),
                                                 ('vector2', TfidfVectorizer(),
                                                  'text')])),
                ('model', LogisticRegression())])

###Checking model is overfit/underfit

In [ ]:
LR_trainscore=pipe.score(xtrain,ytrain)
LR_testscore=pipe.score(xtest,ytest)
ypred=pipe.predict(xtest)
print(f"Train Score: {LR_trainscore}")
print(f"Test Score: {LR_testscore}")

Train Score: 1.0
Test Score: 1.0


###Model Evaluation

In [ ]:
LR_accscore=accuracy_score(ytest,ypred)
LR_precscore=precision_score(ytest,ypred)
LR_recallscore=recall_score(ytest,ypred)
LR_f1score=f1_score(ytest,ypred)
LR_conmatrix=confusion_matrix(ytest,ypred)
print(f"Accuracy Score: {LR_accscore}")
print(f"Precision Score: {LR_precscore}")
print(f"Recall Score: {LR_recallscore}")
print(f"F1 Score: {LR_f1score}")
print(LR_conmatrix)

Accuracy Score: 1.0
Precision Score: 1.0
Recall Score: 1.0
F1 Score: 1.0
[[4733    0]
 [   0 4247]]


##Creating Pipeline for Chaining Preprocessors and **XGBClassifier** Model

In [ ]:
pipe1=Pipeline(steps=[('preprocess',preprocess),
                     ('model',XGBClassifier())])

##Training the **XGBClassifier** Model

In [ ]:
pipe1.fit(xtrain,ytrain)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('encoder', OneHotEncoder(),
                                                  ['subject']),
                                                 ('vector1', TfidfVectorizer(),
                                                  'title'),
                                                 ('vector2', TfidfVectorizer(),
                                                  'text')])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=No...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

###Checking Model is Overfit/Underfit


In [ ]:
XGB_trainscore=pipe1.score(xtrain,ytrain)
XGB_testscore=pipe1.score(xtest,ytest)
ypred1=pipe1.predict(xtest)
print(f"Train Score: {XGB_trainscore}")
print(f"Test Score: {XGB_testscore}")

Train Score: 1.0
Test Score: 0.9998886414253898


###Model Evaluation

In [ ]:
XGB_accscore=accuracy_score(ytest,ypred1)
XGB_precscore=precision_score(ytest,ypred1)
XGB_recallscore=recall_score(ytest,ypred1)
XGB_f1score=f1_score(ytest,ypred1)
XGB_conmatrix=confusion_matrix(ytest,ypred1)
print(f"Accuracy Score: {XGB_accscore}")
print(f"Precision Score: {XGB_precscore}")
print(f"Recall Score: {XGB_recallscore}")
print(f"F1 Score: {XGB_f1score}")
print(XGB_conmatrix)

Accuracy Score: 0.9998886414253898
Precision Score: 1.0
Recall Score: 0.9997645396750647
F1 Score: 0.9998822559755093
[[4733    0]
 [   1 4246]]


###Creating Pipeline for Chaining Preprocessors and Model Algorithm **MultinomialNB**

In [ ]:
pipe2=Pipeline(steps=[('preprocess',preprocess),
                     ('model',MultinomialNB())])

###Training the **MultinomialNB** Model

In [ ]:
pipe2.fit(xtrain,ytrain)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('encoder', OneHotEncoder(),
                                                  ['subject']),
                                                 ('vector1', TfidfVectorizer(),
                                                  'title'),
                                                 ('vector2', TfidfVectorizer(),
                                                  'text')])),
                ('model', MultinomialNB())])

###Checking if the Model is Overfit/Underfit

In [ ]:
mNB_trainscore=pipe2.score(xtrain,ytrain)
mNB_testscore=pipe2.score(xtest,ytest)
ypred2=pipe2.predict(xtest)
print(f"Train Score: {mNB_trainscore}")
print(f"Test Score: {mNB_testscore}")

Train Score: 0.9998051116431873
Test Score: 0.9997772828507795


###Model Evaluation

In [ ]:
mNB_accscore=accuracy_score(ytest,ypred2)
mNB_precscore=precision_score(ytest,ypred2)
mNB_recallscore=recall_score(ytest,ypred2)
mNB_f1score=f1_score(ytest,ypred2)
mNB_conmatrix=confusion_matrix(ytest,ypred2)
print(f"Accuracy Score: {mNB_accscore}")
print(f"Precision Score: {mNB_precscore}")
print(f"Recall Score: {mNB_recallscore}")
print(f"F1 Score: {mNB_f1score}")
print(mNB_conmatrix)

Accuracy Score: 0.9997772828507795
Precision Score: 0.9995293010120028
Recall Score: 1.0
F1 Score: 0.9997645951035782
[[4731    2]
 [   0 4247]]


##Prediction for new input using **LogisticRegression** Model

In [ ]:
new_data = pd.DataFrame({
    'subject': ['politicsNews'],
    'title': ['Government makes a bold claim about economy'],
    'text': ['The government said that the GDP will rise by 20% in the next quarter.']
})
pipe.predict(new_data)

array([1])

The model predicts that new input is  a Real News.

#**CONCLUSION**
##In this **Fake News Detection** project, three different machine learning models were implemented and evaluated: **Logistic Regression, XGBoost Classifier, and Multinomial Naive Bayes**. After training and testing on the dataset, **Logistic Regression** consistently outperformed the other models in terms of **accuracy, precision, recall, and F1-score**.For its strong performance and interpretability, **Logistic Regression** was selected as the final model for deployment in this project.

